The Kmeans algorithm is used here to provide a user-based recommendation system

ref:

https://www.researchgate.net/profile/Lasheng-Yu/publication/314250702_Movies_recommendation_system_using_collaborative_filtering_and_k-means/links/58e4a8c00f7e9bbe9c94dbb4/Movies-recommendation-system-using-collaborative-filtering-and-k-means.pdf

https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=8776969

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt

In [2]:
movies = pd.read_csv('movies_pivot_1.csv')
movies = movies.fillna(0)
model = KMeans()
train,test = train_test_split(movies, train_size= 0.8, shuffle= True)

In [4]:
movies

,Toy Story (1995),GoldenEye (1995),Four Rooms (1995),Get Shorty (1995),Copycat (1995),Shanghai Triad (Yao a yao yao dao waipo qiao) (1995),Twelve Monkeys (1995),Babe (1995),Dead Man Walking (1995),Richard III (1995),...,Mamma Roma (1962),"Sunchaser, The (1996)","War at Home, The (1996)",Sweet Nothing (1995),Mat' i syn (1997),B. Monkey (1998),Sliding Doors (1998).1,You So Crazy (1994),Scream of Stone (Schrei aus Stein) (1991),User Average Rating
0,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.610294
1,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.709677
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.796296
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.333333
4,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.874286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
938,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.265306
939,0.0,0.0,0.0,2.0,0.0,0.0,4.0,5.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.457944
940,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.045455
941,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.265823


In [6]:
import pandas
def user_based_kmeans(data: pandas.DataFrame, user_id: int, number_of_recommendations = 10):
    user_data = pd.DataFrame([data.iloc[user_id]]) 
    user_evaluations = []
    for item in user_data:
        value = user_data.at[user_id, item]
        if value != 0:
            user_evaluations.append([value, item])        
    test_size = np.ceil(len(user_evaluations)) * 0.3
    user_test_data = []
    for i, item in enumerate(user_evaluations):
        if i > test_size:
            break
        elif item[0] > 3:
            user_test_data.append(item)
            user_data.at[user_id, item[1]] = 0
    alt_data = data.drop(index = user_id)
    model = KMeans(n_init=10)
    model.fit(alt_data)
    clusters = model.predict(alt_data)
    alt_data['clusters'] = clusters.tolist()
    user_cluster = model.predict(user_data)
    cluster_value = user_cluster[0]
    user_data['clusters'] = user_cluster.tolist()
    alt_data = pd.concat([alt_data, user_data])
    alt_data = alt_data.loc[alt_data['clusters'] == cluster_value]
    items_means = []
    for item in alt_data:
        if alt_data.at[user_id, item] == 0:
            temp_data = alt_data.copy()
            temp_data = temp_data[temp_data[item] != 0]
            mean = temp_data[item].mean()
            items_means.append([mean, item])    
    items_means.sort(reverse= True)
    comparison_list = []
    for valor_previsto, item1 in items_means:
        for valor_real,item2 in user_test_data:
            if item1 == item2:
                comparison_list.append((valor_real - valor_previsto)**2 )
    comparison_list = np.array(comparison_list)
    comparison_list = comparison_list[~np.isnan(comparison_list)]
    rmse = np.sqrt(np.sum(comparison_list)/len(comparison_list))
    return items_means[0:number_of_recommendations], rmse
    
def pegar_segundo_item(lista):
    lista_aux = []
    for item in lista:
        lista_aux.append(item[1])
   # print(lista_aux)
    return lista_aux

n = 10
matriz = []
for i in range(n):
    recommendations, rmse = user_based_kmeans(data = movies, user_id= 23)
    #print(recommendations)
    #print(f'rmse:{rmse}')
    x = pegar_segundo_item(recommendations)
    print(x)
    matriz.append(x)

['Raging Bull (1980)', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 'Nikita (La Femme Nikita) (1990)', 'Usual Suspects, The (1995)', 'Dead Poets Society (1989)', 'Henry V (1989)', 'Postino, Il (1994)', 'M*A*S*H (1970)', 'Kolya (1996)', 'Field of Dreams (1989)']
['Shall We Dance? (1996)', 'Richard III (1995)', 'Usual Suspects, The (1995)', 'Silence of the Lambs, The (1991)', 'Blade Runner (1982)', 'Postino, Il (1994)', 'Much Ado About Nothing (1993)', 'Searching for Bobby Fischer (1993)', 'Remains of the Day, The (1993)', 'Toy Story (1995)']
['Cry, the Beloved Country (1995)', 'Usual Suspects, The (1995)', 'Grand Day Out, A (1992)', 'Raging Bull (1980)', 'Room with a View, A (1986)', 'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 'Kolya (1996)', 'Searching for Bobby Fischer (1993)', 'Postino, Il (1994)', "Mr. Holland's Opus (1995)"]
['Paradise Lost: The Child Murders at Robin Hood Hills (1996)', 'Ice Storm, The (1997).1', 'Four Rooms (1995)', 'Henry V (1989)', 'Ragin

In [7]:
from collections import Counter

matriz = np.array(matriz)
aux = matriz.flatten()
count = Counter(aux)
count.most_common(8)

[('Usual Suspects, The (1995)', 9),
 ('Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)', 8),
 ('Postino, Il (1994)', 7),
 ('Kolya (1996)', 7),
 ('Raging Bull (1980)', 6),
 ('Henry V (1989)', 6),
 ('Room with a View, A (1986)', 6),
 ('Nikita (La Femme Nikita) (1990)', 4)]